# **Classification**
data가 여러 곳에 산점이 되어있을 때 grouping, 나눠주는 것을 의미한다

i.e. Grouping 할 수 있는 문제들은 Classification 문제라고 한다 

ML, NLP, Deep-Learning 문제는 모두 Classfication 또는 Regression으로 해결할 수 있다

## **import Data**

In [42]:
import pandas as pd 
import numpy as np 

In [43]:
df = pd.read_csv("../input/hbpark-titanic-data/titanic.csv")
df[:5]

## **1. 모델에 사용할 컬럼(column, feature) 결정**

In [44]:
len(df)

In [45]:
df.columns

Classification 문제이기 때문에 'Survived'(생존여부)를 맞추도록 하자

In [47]:
X_column = ['Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Embarked']
y_column = 'Survived' # 정답 값, y, true value, target value, label

정답값 분포를 확인해 보자 -> 0 or 1 

클래스(0,1) 예측 -> Classfication 문제가 된다

In [48]:
print(df['Survived'].unique()) # 0 사망, 1 생존 
print(df['Survived'].nunique())

최종적으로 모델에 사용할 컬럼 (column, feature) 결정

In [49]:
input_data = df[X_column + [y_column]].copy() # Memory
input_data

## **2. 모델에 입력할 데이터 타입 변환**

numeric value 숫자만 가능하다, int, float

In [50]:
input_data.dtypes 

In [51]:
# categorical columns extraction
# cat_cols = ['Sex', 'Embarked']
cat_cols = input_data.columns[input_data.dtypes == object]
cat_cols

In [52]:
# label encoding (integer encoding)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for c in cat_cols : 
    input_data[c] = le.fit_transform(list(input_data[c])) # list() 로 씌우면 결측치 자동 처리

input_data

In [53]:
# Embarked의 마지막 값들을 확인할 수 있음  
print(le.classes_)
#결측치 변환값을 확인 
print(input_data['Embarked'].unique())

In [54]:
input_data.dtypes

모델에 결측치가 있으면 학습을 하지 못하기 때문에 결측치를 채워줘야 한다

In [69]:
# 결측치에 대해 graph로 보여준다 
import missingno as msno
msno.matrix(input_data, figsize=(15,8))

In [70]:
# 결측치가 있는 column을 찾는 것 
input_data.columns[input_data.isna().any()].tolist()

In [73]:
# 결측치 처리 -> 다른 좋은 모델들은 결측치 처리 안하기도 함 
input_data_base = input_data.copy() # 임시 저장 

In [75]:
# 결측치 처리 
input_data = input_data.fillna(-1) # nan -> -1

In [77]:
# 결측치가 처리 완료 확인
input_data.columns[input_data.isna().any()].tolist()

In [80]:
# 결측치 처리 완료 확인
import missingno as msno
msno.matrix(input_data, figsize=(15,8))

결측치 처리는 배경지식을 토대로 값을 채워넣을 수 있다

이를테면 다른 값들과 겹치지 않는 -1 값을 넣는 방법이 있다 

좋은 모델 xgb, lgbm, catboost는 결측치를 알아서 처리한다 

알아서 처리할 경우 -1을 넣는 것보다는 더 좋은 성능을 가지기도 한다 

## **3. 학습 데이터 준비**
모델 학습을 위해 필요한 세 가지 데이터 
1. train data 학습 
   * train data 
   * validation data, 학습 하는 도중에 실행하는 성능평가 
2. test data 모델 성능 평가

Our Total Data = 20 개 

16개 -> Train -> train(12), validation(4) 

4개 -> test

**Classification issue** 
분석 -> 데이터 분포가 중요함 

**Classification에서 데이터의 분포를 균등하게 가공하는 것은 굉장히 중요하다!**


In [89]:
# Survived 분포 확인 
# 0 사망 549명 
# 1 생존 342명 
df[y_column].value_counts() # unique() + nunique() 동시 출력

만약 최악의 경우 

0 50명 사망 

1 나머지 다 생존 

=>model 학습이 이상해질 수 있음 

In [90]:
# 데이터 -> train/valid , test
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data[X_column], # X 입력 데이터 -> tr_val_X, test_X
    input_data[y_column], # Y 정답 데이터 -> tr_val_y, test_y
    test_size = 0.2,
    #train_size = 13, 
    shuffle=True,
    random_state=42,
    stratify=input_data[y_column] # 분포, 모델성능에 굉장히 중요한 요소! Classification 문제에서는 필수!
)

In [91]:
tr_val_data = pd.concat([tr_val_X, tr_val_y], axis=1)
tr_val_data

In [92]:
# train / valid data로 split 
train_X, valid_X, train_y, valid_y = train_test_split(
    tr_val_data[X_column], # X 입력 데이터 - > train_X, valid_X
    tr_val_data[y_column], # Y 정답 데이터 -> train_y, valid_y
    test_size = 0.2, 
    shuffle=True,
    random_state=42,
    stratify=tr_val_data[y_column]
)

In [93]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))
len(train_X) + len(valid_X) + len(test_X) # 20

In [94]:
train_y.value_counts() # 예측값과 실제값이 비슷함 

## **4. 본격적인 모델 학습의 시작!**
Decision Tree, Random Forest, XGB, CATBOOST

In [96]:
from sklearn.ensemble import RandomForestClassifier

#clf = RandomForestClassifier -> clf = classification 
model_rf = RandomForestClassifier()
model_rf.fit(train_X, train_y)

In [98]:
display(test_X, test_y)

In [97]:
# 정답 값을 의미, 이해하기 쉽게 변수명 설정
true_y = test_y
true_y

In [105]:
# classification prediction (예측)
pred_y_proba = model_rf.predict_proba(test_X) # column이 2차원으로 변형됨 -> 0일 확률, 1일 확률 
#pred_y = model_rf.predict(test_X)
pred_y_proba

In [109]:
pred_y = np.argmax(pred_y_proba, axis=1) # 0 과 1 중 확률이 높은 index를 추출한다
pred_y

In [111]:
# 정답값(true_y) 와 예측값(pred_y)를 한눈에 보기 위한 function
df_result = pd.DataFrame(list(zip(test_y, pred_y)), columns=['true_y', 'pred_y'])
df_result

mse, rmse ---> regression model의 평가 방법 

classification_report, confusion_matrix ----> classification의 평가 방법(모델)

In [113]:
from sklearn.metrics import classification_report, confusion_matrix

# f1-score : precision 과 recall의 average --> 잘 예측한 정도
# accuracy : model accuracy --> 사망 데이터를 더 잘 예측하는 것으로 보인다 => 
print(classification_report(true_y, pred_y))

**만약, 특정 label에 대해서만 accuracy가 떨어진다면 의심해봐야 한다 ==> 학습 평가 지표**

In [116]:
confusion_matrix(true_y, pred_y)
#         정답 0 정답 1 
# 예측 0 
# 예측 1 
# ctrl + / : 모두 주석 처리 

In [ ]:
# 매출액 돈 예측은 
# Regression으로 해결해야 함 
# model 성능은 잘 맞춰서 Classification이 더 좋긴 함  
# 그래서 Classification 문제로 변경해서 풀 수 있음 
# 0~1000 -> 0  
# 1001~2000 -> 1 로 바꿔서 

## **Ex_xgboost_XGBClassifier**

In [117]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(learning_rate=0.1)
model_xgb.fit(train_X, train_y, eval_set=[(valid_X, valid_y)])

In [118]:
# classification prediction (예측)
xgb_pred = model_xgb.predict(test_X)
#pred_y = model_rf.predict(test_X)
xgb_pred

In [120]:
print(classification_report(true_y, xgb_pred))

In [121]:
model_rf.feature_importances_

In [122]:
# 중요도 순으로 자동 Sorting
pd.Series(model_rf.feature_importances_, index=train_X.columns).sort_values(ascending=False)

In [123]:
# 중요도 순으로 자동 Sorting
pd.Series(model_xgb.feature_importances_, index=train_X.columns).sort_values(ascending=False)

RandomForst, XGB 모델을 사용하였는데 

XGB 모델이 더 좋았다 -> 제일 좋은 XGB 모델을 사용하였다 -> 라는 식으로 report를 작성하면 될 것 같습니다 

* 각 COLUMN 별 feature importance 에 대한 report...

feature importance에 대해서 report를 하고 싶다면 

<model_rf.feature_importances_>

<pd.Series(model_rf.feature_importances_, index=train_X.columns).sort_values(ascending=False)>

중요도가 낮다고 해서 빼고 학습하면 안좋으므로 가능한 많은 feature를 사용하는 것이 좋다

Age에 결측치를 넣어주었으므로 Age를 가장 중요한 feature로 본 model에 대한 신뢰도는 낮을 것으로 예상된다 
